In [158]:
import numpy as np
import pandas as pd

### 1번 데이터프레임 처리과정 (구별 인구데이터)

In [159]:
# 서울시 구별 주민등록정보 데이터를 s로 이름
s = pd.read_csv('서울시주민등록인구(구별)통계.txt', skiprows=2, sep='\t')
s.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
1,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
2,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"
3,2021.2/4,용산구,"111,650","240,665","116,858","123,807","226,378","108,948","117,430","14,287","7,910","6,377",2.03,"38,884"
4,2021.2/4,성동구,"134,542","295,767","143,927","151,840","289,162","141,042","148,120","6,605","2,885","3,720",2.15,"45,591"


In [160]:
# 1번데이터 전처리    #컬럼별 결측값 없음을 확인할 수 있다.
# s.isna().sum()      #컬럼별 결측값.
s.isna().sum().sum()  #전체 결측값. 확인차.
# s.info()  #인구 값들이 문자열이라 int 변환및   1000단위의  컴마 , 제거할것.

0

In [161]:
s.columns

Index(['기간', '자치구', '세대', '계', '남자', '여자', '계.1', '남자.1', '여자.1', '계.2',
       '남자.2', '여자.2', '세대당인구', '65세이상고령자'],
      dtype='object')

In [162]:
s = s[['자치구', '계', '계.2', '65세이상고령자']]   # 컬럼추출
# s.head(3)

In [163]:
s.columns = ['구별', '인구수', '외국인', '고령자']    # 컬럼이름 변경
# s.head(3)

In [164]:
# , 없애기 및 정수변환 / 주의할점은 s.columns[1:] 의미.
for column in s.columns[1:]:
    s[column] = s[column].apply(lambda x : int(x.replace(',','')))
# s.info()  #정수로 잘 변경됨 확인

In [165]:
# 외국인비율(%) 및 고령자비율(%) 추가
# 값 자릿수 고려. 소숫점이하 2자리까지. 
s['외국인비율(%)'] = np.round((s.외국인 / s.인구수) * 100, 2)
s['고령자비율(%)'] = np.round((s.고령자 / s.인구수) * 100, 2)
# 이거 한번에 되나? #둘이 결합하는 방법
# s.head(3)

In [166]:
s.drop(columns= ['외국인','고령자'],inplace=True)
s.head(3)

,구별,인구수,외국인비율(%),고령자비율(%)
0,합계,9795426,2.34,16.10
1,종로구,155106,5.85,17.80
2,중구,132259,6.99,18.13


### 2번 데이터프레임 처리과정 (구별 cctv 데이터)

In [167]:
# 데이터 불러오기 및 전처리
# 세퍼레이션은 컴마, 로 되어있음
# 외계어 나와서 인코딩 'euc-kr' 적용
c = pd.read_csv('서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', sep=',', encoding='euc-kr', skiprows=1)
c.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


- null data 추적
- 천단위 구분기호 , 제거 및 정수변경
- 필요컬럼 추리기 이름변경 드랍 등 

In [168]:
# c.isnull()
c.isna().sum().sum()
# 결측치 없다. info는 생략. 

0

In [169]:
for column in c.columns[1:]:
    c[column] = c[column].apply(lambda x : int(x.replace(',','')))
c.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76


In [170]:
c.columns

Index(['구분', '총계', '2012년 이전', '2012년', '2013년', '2014년', '2015년', '2016년',
       '2017년', '2018년', '2019년', '2020년', '2021년'],
      dtype='object')

In [171]:
# 최근증가율 컬럼 추가. 다만 %라고 생각하기엔 애매해서 '최근증가율'로 마무리 
c['최근증가율'] = ((( c['2018년'] + c['2019년'] + c['2020년'] ) / (
                                 c['2012년 이전'] + c['2013년'] + c['2014년'] + c['2015년'] + c['2016년'] +c['2017년'] ))*100).round(2)
# c.head()

In [172]:
c = c[['구분', '총계', '최근증가율']]
c.head()

,구분,총계,최근증가율
0,계,77032,81.90
1,종로구,1772,20.49
2,중 구,2333,87.74
3,용산구,2383,83.06
4,성동구,3602,48.65


In [173]:
# df.rename(columns={'Before':'After'}) # 열 선택하여 바꿀때 
# df.columns = ['col', 'col', 'col']    # 전체컬럼이름 바꿀때는 요게 편하다
c.columns = ['구별', 'CCTV댓수', '최근증가율']
c.head(3)

,구별,CCTV댓수,최근증가율
0,계,77032,81.90
1,종로구,1772,20.49
2,중 구,2333,87.74


In [174]:
# 구이름 공백지우기 # 중구 데이터 하나만 '중 구'로 띄어쓰기 되어있어서 처리 필요
c['구별'] = c.구별.apply(lambda x: x.replace(' ',""))
c.head()

,구별,CCTV댓수,최근증가율
0,계,77032,81.90
1,종로구,1772,20.49
2,중구,2333,87.74
3,용산구,2383,83.06
4,성동구,3602,48.65


In [178]:
# c.drop([0], axis=0, inplace=True) # 첫번째 행 계 부분 드랍
# c.head()

,구별,CCTV댓수,최근증가율
1,종로구,1772,20.49
2,중구,2333,87.74
3,용산구,2383,83.06
4,성동구,3602,48.65
5,광진구,2588,117.87


### 3. 공통 컬럼 및 인덱싱 확인 merge 이용해 붙이기
 - cctv 데이터인 c에 서울시 인구데이터 s 를 붙일것 : 주요데이터가 CCTV댓수

In [176]:
cctv = pd.merge(c, s)

In [177]:
# 교재 63p처럼 '구별'컬럼 인덱스로 만들기
cctv.set_index('구별')
cctv

,CCTV댓수,최근증가율,인구수,외국인비율(%),고령자비율(%)
구별,,,,,
종로구,1772,20.49,155106,5.85,17.80
중구,2333,87.74,132259,6.99,18.13
용산구,2383,83.06,240665,5.94,16.16
성동구,3602,48.65,295767,2.23,15.41
광진구,2588,117.87,355306,3.61,14.30
동대문구,2497,37.91,352259,4.04,17.39
중랑구,3296,195.21,394679,1.19,17.78
성북구,3958,98.49,443206,2.15,16.60
강북구,2462,446.95,306093,1.13,20.71
